## Imports

In [1]:
from transformers import BertTokenizer

# from torch.utils.data import Dataset
from models.CustomDatasetBinary import CustomDatasetBinary
from torch.utils.data import DataLoader

from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Datasets & loaders

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# starting w/ MD dataset
df = pd.read_csv('../data/MD_LoD.csv', index_col = 0)
df

# exclude labels == 2 (slightly agreement)
df = df[df['disagreement_lev'] != 2]

train_set = CustomDatasetBinary(df[df['split'] == 'train'], tokenizer, label_col = 'disagreement_lev')
val_set = CustomDatasetBinary(df[df['split'] == 'dev'], tokenizer, label_col = 'disagreement_lev')
test_set = CustomDatasetBinary(df[df['split'] == 'test'], tokenizer, label_col = 'disagreement_lev')

print(f'len: \n\tTrain: {len(train_set)}\n\tVal: {len(val_set)}\n\tTest: {len(test_set)}')

train_loader = DataLoader(train_set, batch_size = 8, shuffle=True)
val_loader = DataLoader(val_set, batch_size = 8, shuffle=True)
test_loader = DataLoader(test_set, batch_size = 8, shuffle=True)

len: 
	Train: 4662
	Val: 787
	Test: 2148


## Model

In [3]:
from models.TransformerBinaryClassifier import TransformerBinaryClassifier

from transformers import BertModel
bert_model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# input of the model: 
#   input_ids
#   attention_mask
model = TransformerBinaryClassifier(
    model = bert_model,
    dropout_rate = .2,
)
inp, lbl = next(iter(train_loader))
out = model(
    inp['input_ids'],
    inp['attention_mask'],
)
print(out.shape)
out.reshape(len(out)).shape


torch.Size([8, 1])


torch.Size([8])

In [6]:
from models.utils import train_loop
import torch.nn as nn

from torch.optim import Adam
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr = 1e-4)

hystory = train_loop(
    epochs = 5, 
    model = model, 
    device = 'cuda', 
    train_loader = train_loader, 
    val_loader = val_loader, 
    optimizer = optimizer, 
    criterion = criterion,
    log_freq = 50, 
    name = 'test0',
)

-- Epoch 01 --
   01) [train,    50] 	loss: 0.696
   01) [train,   100] 	loss: 0.690
   01) [train,   150] 	loss: 0.680


KeyboardInterrupt: 